In [ ]:
# The League of Arbitrary Rules Reddit Eliminator League 20/21
# League Rules:
# 1. finish GW with round number (10, 20, 30..)
# 2. finish GW less than 10 points
# 3. total points tally of 1000, 2000, 3000
# 4. if total eliminated is more than 12%, then no eliminations that week
# 5. eliminator phase stops at GW34
# Link to google sheet:  https://docs.google.com/spreadsheets/d/1Af20eCuLPr6TktC-ISJSiMUpGnNcytpLCzf7AEoz5iw/edit?usp=sharing

In [ ]:
########################################################################################################
#STEP 1: Creates a list of the URLs for the league
#Good
#Gathers teamID(entry)teamName(entry_name) manager first and last name and time they joined the league
#import important tools
import requests
import pandas as pd
import numpy as np

#define the Urls from which you will pull the data. The range for i is based on the number of pages the league takes up
#50 teams per page for the 2020/2021 season and 10 pages of participants 
leagueUrls=[]
leagueCode=269830
for i in range(1,16):
    fullUrl  = 'https://fantasy.premierleague.com/api/leagues-classic/'+str(leagueCode)+'/standings/?page_new_entries=&page_standings='+ str(i)
    leagueUrls.append(fullUrl)

#in case login is required but I did not fill this out apparently
session = requests.session()
url = 'https://users.premierleague.com/accounts/login/'
payload = {
 'password': 'YOUREMAILMAYBE',
 'login': 'YOURPASSWORDMAYBE_COULDBEREVERSED',
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}
session.post(url, data=payload)
print(leagueUrls)

In [ ]:
########################################################################################################
#STEP 2: Grab player info and create a list of lists with this information
#time to grab the variables for the league teams
lInfo=[]
leagueInfo=[]
for leagueUrl in leagueUrls:
    r = requests.get(leagueUrl)
    json = r.json()
    Test=(json['standings'])
    team=Test['results']
    for item in team:
        teamID=item['entry']
        teamName=item['entry_name']
        mgrName=item['player_name']
        linfo=[teamID,teamName,mgrName]
        lInfo.append(linfo)
    #print(teamInfo)
    #create dataframe and append together for each Url
    leagueInfo.append(pd.DataFrame(team)) 
#concatenate the DFs together
result=pd.concat(leagueInfo)
#export to excel- Only run for the first GW to get list of GW people
#result.to_excel(r'C:\OUTFILELOCATION', index = False, sheet_name="SEASON BEGINNING")

In [ ]:
########################################################################################################
#Step 3: Preview list of lists to confirm its structure
print(lInfo)

In [ ]:
########################################################################################################
#Step 4: Create GW URLs to scrape player data from
#AFTER SEASON STARTS
#next, grab the score for each team in the league for each gameweek
#first, must create links for all of the players in the league and append to end of list

################
##Edit GW HERE##
################
gameweek=2
for r in lInfo:
    twUrl='https://fantasy.premierleague.com/api/entry/'+str(leagueCode)+'/event/'+str(gameweek)+'/picks/#/'
    r.append(twUrl)

In [ ]:
########################################################################################################
#Step 5: Gather gw score and total score. hits are counted in this league
#lInfo should be structured as a list of lists. the inner lists should have 4 elements: teamid,teamname,managername,url
#because this is a list of lists and not a dictionary, the elements will be unnamed but that is what each element actually is
#now that I have all the links created, need to gather the gw scores and overall score totals
for lInfos in lInfo:
    gwteamUrl= lInfos[3]
    r = requests.get(gwteamUrl)
    json1 = r.json()
    json1.keys()
    gwscoreinfo=(json1['entry_history'])
    gwscore1=gwscoreinfo['points']
    totalscore=gwscoreinfo['total_points']
    hits=gwscoreinfo['event_transfers_cost']
    gwscore=gwscore1-hits
    #note, if you want to exclude hits from the gwscore, replace the line of code below with 'lInfos.append(gwscore1)'
    lInfos.append(gwscore)
    lInfos.append(totalscore)

In [ ]:
########################################################################################################
#Step 6: Export GW results to excel and create new sheet for this GW specifically. Make sure gameweek variable is updated above!
gwrs=pd.DataFrame(lInfo)
with pd.ExcelWriter(r'C:\PATH\Arbitrary.xlsx', engine='openpyxl', mode='a') as writer: 
     gwrs.to_excel(writer, sheet_name=str(gameweek), index=False, header=False) 

In [ ]:
########################################################################################################
#Step 7: time to determine who is eliminated
import math
#calculate threshold for elimination of 12 percent
thresh= .12*len(lInfo)
#create list of possibly eliminated managers that conforms to managername and team name in parentheses
#because that is the format required for automated manager elimination Steps 8-12 
#list considers rules of GW score <10, divisible by 10, or total score divisible by 1000
s = [ str(mgrName_)+" ("+str(teamName_)+")" for (teamID_, teamName_, mgrName_, URL_, gwscore_, totscore_) in lInfo if int(gwscore_) < 10 or int(gwscore_)%10==0 or int(totscore_)%1000==0 ]
#number of managers in league this week
print(len(lInfo))
#number of tentatively eliminated managers
print(s)
#if tentatively eliminate number of managers less than safety threshold, output that managers are eliminated
print('Thresh: '+str(thresh)+';Tentatively eliminated: '+str(len(s)))
if len(s)<=thresh:
    print('eliminations this week')
else:
    print('No eliminations this week')

In [ ]:
########################################################################################################
#automated removal of FPL managers eliminated from league (list 's' from Step 7 contains these managers in the proper format)
#if output from Step 7 states eliminations this week, then proceed with the remaining code. If not, then your work is finished this GW

#Step 8: Removing players from league using selenium
# Selenium import
pip install selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [ ]:
########################################################################################################
#Step 9: open Selenium controlled web browser and inject javascript
driver= webdriver.Chrome(r'C:\PATH\chromedriver.exe')
def goUrl(url):
    #navigate to url
    driver.get(url) 
    # load jquery 
    with open(r'C:\PATH\jquery-3.2.1.js', 'r') as jquery_js:         
        # read and parse jquery lib
        driver.execute_script( jquery_js.read() ) 
    return# helper function that navigates to a page, and then reads jquery from disk

In [ ]:
########################################################################################################
#Step 10: Navigate to fantasy page requiring login
#use goUrl function to navigate to url and inject jquery
goUrl('https://fantasy.premierleague.com/leagues/'+str(r[0])+'/admin/c')


In [ ]:
########################################################################################################
#Step 11: have selenium log you into the FPL site
driver.implicitly_wait(1)
# Find the HTML element for username, and fill in your name
myName = 'EMAIL'
myPassword = 'PASSWORD'
# fill out username and password using Selenium
#use the ID field. Notes for future self: inspect page, then find the specific field. Right click and copy JS, it gave the loginUsername/Password 
driver.execute_script("$('#loginUsername').val('" + myName + "')")
driver.execute_script("$('#loginPassword').val('" + myPassword + "')")

# click submit button- Notes for future self: I determined the path via inspect the button and copy xpath value
driver.find_element_by_xpath('//*[@id="root"]/div[2]/div/div/div[1]/form/div[3]/button').click()

In [ ]:
########################################################################################################
#Step 12: remove eliminated managers
#go through list of losers in s and manually remove them from the league
for loser in s:
    # use goUrl function to navigate to url and inject jquery into the admin page
    goUrl('https://fantasy.premierleague.com/leagues/'+str(leagueCode)+'/admin/c')
    time.sleep(2)
    #send first name
    driver.find_element_by_xpath('//*[@id="downshift-0-label"]').send_keys(loser)
    #send space bar to get FPL site to create dropdown box
    driver.find_element_by_xpath('//*[@id="downshift-0-label"]').send_keys(Keys.SPACE)
    #wait until drop down appears
    time.sleep(2)
    text=[]
    i=0
    #maybe try grabbing a list of all the drop down options text and clicking if text matches loser
    #true statement would be loser != text <---does not work
    while str(loser) != str(text):
        for i in range(0,30):
            xpathy='//*[@id="downshift-0-item-'+ str(i)+'"]'
            try:
                text = driver.find_element_by_xpath(xpathy).text
                if str(text)==str(loser):
                    driver.find_element_by_xpath(xpathy).click()
                    time.sleep(1)
                    driver.find_element_by_xpath('//*[@id="root"]/div[2]/div/div/div/form[2]/div[2]/button').click()
                    print(loser)
                    break
            except:
                print(loser + ' failed to remove')
                i=i+1
                break
        #if fails through 10 iterations, then hopefully it breaks the code and prevents loop
        if i>10:
            print(loser + ' failed to remove')
            break      